In [6]:
import pandas as pd          
from surprise import Dataset, Reader, SVD  
import os

In [7]:
os.environ['KAGGLE_CONFIG_DIR'] = "kaggle.json"

In [8]:
libro_ratings = pd.read_csv('data/Books_rating.csv')
libro_data = pd.read_csv('data/Books_data.csv')

In [9]:
print(libro_ratings.head())
print(libro_data.head())

           Id                           Title  Price         User_id  \
0  1882931173  Its Only Art If Its Well Hung!    NaN   AVCGYZL8FQQTD   
1  0826414346        Dr. Seuss: American Icon    NaN  A30TK6U7DNS82R   
2  0826414346        Dr. Seuss: American Icon    NaN  A3UH4UZ4RSVO82   
3  0826414346        Dr. Seuss: American Icon    NaN  A2MVUWT453QH61   
4  0826414346        Dr. Seuss: American Icon    NaN  A22X4XUPKF66MR   

                          profileName review/helpfulness  review/score  \
0               Jim of Oz "jim-of-oz"                7/7           4.0   
1                       Kevin Killian              10/10           5.0   
2                        John Granger              10/11           5.0   
3  Roy E. Perry "amateur philosopher"                7/7           4.0   
4     D. H. Richards "ninthwavestore"                3/3           4.0   

   review/time                                   review/summary  \
0    940636800           Nice collection of Julie Strai

In [10]:
popularidad = libro_ratings[['Title', 'User_id', 'review/score']].copy()
print(popularidad.isnull().sum())

Title              208
User_id         561787
review/score         0
dtype: int64


In [11]:
popularidad.dropna(inplace=True)
print(popularidad.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2438018 entries, 0 to 2999998
Data columns (total 3 columns):
 #   Column        Dtype  
---  ------        -----  
 0   Title         object 
 1   User_id       object 
 2   review/score  float64
dtypes: float64(1), object(2)
memory usage: 74.4+ MB
None


In [12]:
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(popularidad[['User_id', 'Title', 'review/score']], reader)

In [13]:
trainset = data.build_full_trainset()

In [14]:
entrenamiento = SVD()
entrenamiento.fit(trainset)

In [15]:
def recomendacion_de_libros(titulo, entrenamiento, df, n=3):
    user_ids = df['User_id'].unique()
    book_id = df[df['Title'] == titulo].iloc[0]['Title']
    predictions = [entrenamiento.predict(user_id, book_id) for user_id in user_ids]
    top_n = sorted(predictions, key=lambda x: x.est, reverse=True)[:n]
    recomendacion = [pred.iid for pred in top_n]
    
    return recomendacion

titulo = "Dr. Seuss: American Icon"
recomendacion = recomendacion_de_libros(titulo, entrenamiento, popularidad)
print(recomendacion)

<function recomendacion_de_libros at 0x000001C695CC31F0>
